---
#**EXTRACTION METHOD**
---

# Install necessary dependencies

In [1]:
import nltk
import numpy as np
import pandas as pd
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Get Text Document

We use the description of a very popular role-playing game (RPG) Skyrim from
Bethesda Softworks for summarization. 

In [2]:
DOCUMENT = """
The Elder Scrolls V: Skyrim is an action role-playing video game developed by Bethesda Game Studios and published by Bethesda Softworks.

It is the fifth main installment in The Elder Scrolls series, following The Elder Scrolls IV: Oblivion.
"""

In [3]:
import re
DOCUMENT = re.sub(r'\n|\r', ' ', DOCUMENT)
#DOCUMENT = re.sub(r' +', ' ', DOCUMENT)
DOCUMENT = DOCUMENT.strip()

In [4]:
print(DOCUMENT)

The Elder Scrolls V: Skyrim is an action role-playing video game developed by Bethesda Game Studios and published by Bethesda Softworks.  It is the fifth main installment in The Elder Scrolls series, following The Elder Scrolls IV: Oblivion.


Sentences Collection

In [5]:
sentences = nltk.sent_tokenize(DOCUMENT)
len(sentences)
sentences

['The Elder Scrolls V: Skyrim is an action role-playing video game developed by Bethesda Game Studios and published by Bethesda Softworks.',
 'It is the fifth main installment in The Elder Scrolls series, following The Elder Scrolls IV: Oblivion.']

# Basic Text pre-processing

In [10]:
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document) #Function Def Vectorize
norm_sentences = normalize_corpus(sentences)
norm_sentences[:3]

array(['elder scrolls v skyrim action roleplaying video game developed bethesda game studios published bethesda softworks',
       'fifth main installment elder scrolls series following elder scrolls iv oblivion'],
      dtype='<U113')

# Text Representation with Feature Engineering

We will be vectorizing our normalized sentences using the TF-IDF feature engineering
scheme. We keep things simple and don’t filter out any words based on document
frequency. But feel free to try that out and maybe even leverage n-grams as features.

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
dt_matrix = tv.fit_transform(norm_sentences)
dt_matrix = dt_matrix.toarray()
dt_matrix

array([[0.24244659, 0.48489319, 0.24244659, 0.17250275, 0.        ,
        0.        , 0.48489319, 0.        , 0.        , 0.        ,
        0.        , 0.24244659, 0.24244659, 0.17250275, 0.        ,
        0.24244659, 0.24244659, 0.24244659, 0.24244659],
       [0.        , 0.        , 0.        , 0.42808473, 0.30082908,
        0.30082908, 0.        , 0.30082908, 0.30082908, 0.30082908,
        0.30082908, 0.        , 0.        , 0.42808473, 0.30082908,
        0.        , 0.        , 0.        , 0.        ]])

In [16]:
td_matrix = dt_matrix.T
td_matrix
print(td_matrix.shape)

(19, 2)


In [14]:
vocab = tv.get_feature_names()
vocab

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['action',
 'bethesda',
 'developed',
 'elder',
 'fifth',
 'following',
 'game',
 'installment',
 'iv',
 'main',
 'oblivion',
 'published',
 'roleplaying',
 'scrolls',
 'series',
 'skyrim',
 'softworks',
 'studios',
 'video']

In [12]:

print(td_matrix.shape)
pd.DataFrame(np.round(td_matrix, 2), index=vocab).head(10)

(19, 2)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,0,1
action,0.24,0.00
bethesda,0.48,0.00
developed,0.24,0.00
elder,0.17,0.43
fifth,0.00,0.30
following,0.00,0.30
game,0.48,0.00
installment,0.00,0.30
iv,0.00,0.30
main,0.00,0.30
